Import Tensorflow and open a session.

In [2]:
%run 'Ising.py'

### Determinant
Calculate matrix determinant

In [4]:
W = tf.constant([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.matrix_determinant(W)
tf.Session().run(f)

9.0

Train by minimizing the ln(det(W)). TF knows how to do the gradient.

In [5]:
W = tf.Variable([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.log(tf.matrix_determinant(W))
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(f)
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train)
    print(sess.run(f))

2.06162
1.91632
1.75986
1.5904
1.4056


### Sparse Tensor
Sparse tensor must be converted to dense when passed to the determinant op.

In [337]:
W = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1., 2.], dense_shape=[2, 2])
W_dense = tf.sparse_tensor_to_dense(W)
f = tf.log(tf.matrix_determinant(W_dense))
sess.run(f)

0.69314718

## Entanglement
### Entanglement Regions

In [43]:
list(Region({1,2},4).equivalences())

[{2, 3}, {0, 1}, {1, 2}, {0, 3}]

In [44]:
Region({1,2},4).config()

array([ 1.000, -1.000, -1.000,  1.000])

In [42]:
list(Regions(8).consecutives())

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7}]

In [41]:
list(Regions(8).representatives())

[set(),
 {0},
 {0, 1},
 {0, 2},
 {0, 3},
 {0, 4},
 {0, 1, 2},
 {0, 1, 3},
 {0, 1, 4},
 {0, 2, 4},
 {0, 2, 5},
 {0, 1, 2, 3},
 {0, 1, 2, 4},
 {0, 1, 2, 5},
 {0, 1, 3, 5},
 {0, 1, 3, 6},
 {0, 1, 4, 5},
 {0, 2, 4, 6}]

### Entanglement Entropy
Entanglement entropy of free fermion.
$$H=\sum_{i}(1+(-)^i m)\mathrm{i}\chi_{i}\chi_{i+1}$$

In [745]:
system = FreeFermion(mass=1.e-10, size=8)
system.S([0,1,2,3,4])

0.84523244468672853

### Test

In [709]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=256), CylindricalModel(Lx = 8, Lz = 2))
training_set = machine.training_set('representatives')
writer = machine.writer()
writer.add_graph(sess.graph)
sess.run(machine.model.initialize)

In [712]:
for i in range(2000):
    machine.step += 1
    sess.run(machine.model.train, training_set)
    writer.add_summary(sess.run(machine.model.summarize, training_set), machine.step)
sess.run([machine.model.MSE, machine.model.L2, machine.model.cost], training_set)

[7.0578566877442751e-06, 0.0, 7.0578566877442751e-06]

### Square Lattice Adjacency Matices

In [297]:
%run 'Ising.py'
latt = SSLattice(width = 16, depth = 5, pattern = [-1,-1,1])
latt.print_structure(upto = 'cell')

depth =  5
Chain[0]
├ Cell[0, 0] IR
├ Cell[0, 1] IR
├ Cell[0, 2] IR
├ Cell[0, 3] IR
├ Cell[0, 4] IR
├ Cell[0, 5] IR
├ Cell[0, 6] IR
├ Cell[0, 7] IR
├ Cell[0, 8] IR
├ Cell[0, 9] IR
├ Cell[0, 10] IR
├ Cell[0, 11] IR
├ Cell[0, 12] IR
├ Cell[0, 13] IR
├ Cell[0, 14] IR
├ Cell[0, 15] IR
Chain[1]
├ Cell[1, 0] UV
├ Cell[1, 1] UV
├ Cell[1, 2] IR
├ Cell[1, 3] UV
├ Cell[1, 4] UV
├ Cell[1, 5] IR
├ Cell[1, 6] UV
├ Cell[1, 7] UV
├ Cell[1, 8] IR
├ Cell[1, 9] UV
├ Cell[1, 10] UV
├ Cell[1, 11] IR
├ Cell[1, 12] UV
├ Cell[1, 13] UV
├ Cell[1, 14] IR
├ Cell[1, 15] UV
├ Cell[1, 16] UV
├ Cell[1, 17] IR
├ Cell[1, 18] UV
├ Cell[1, 19] UV
├ Cell[1, 20] IR
├ Cell[1, 21] UV
├ Cell[1, 22] UV
├ Cell[1, 23] IR
Chain[2]
├ Cell[2, 0] UV
├ Cell[2, 1] UV
├ Cell[2, 2] IR
├ Cell[2, 3] UV
├ Cell[2, 4] UV
├ Cell[2, 5] IR
├ Cell[2, 6] UV
├ Cell[2, 7] UV
├ Cell[2, 8] IR
├ Cell[2, 9] UV
├ Cell[2, 10] UV
├ Cell[2, 11] IR
Chain[3]
├ Cell[3, 0] UV
├ Cell[3, 1] UV
├ Cell[3, 2] IR
├ Cell[3, 3] UV
├ Cell[3, 4] UV
├ Cell[3, 5] IR
Cha

### Test

In [298]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=128), IsingModel(SSLattice(width = 16, depth = 5, pattern = [-1,-1,1])))
machine.initialize()

In [328]:
for i in range(5):
    print(machine.train(1, check = 1))

(0.00054776735837535544, array([ 0.186,  0.173,  0.114,  0.054]))
(0.00054717380290481585, array([ 0.186,  0.173,  0.114,  0.054]))
(0.00054658386136046929, array([ 0.186,  0.173,  0.114,  0.054]))
(0.0005459982481037859, array([ 0.186,  0.172,  0.114,  0.054]))
(0.00054541739671703409, array([ 0.186,  0.172,  0.114,  0.054]))


In [335]:
machine.data.source[machine.model.Ssys]/machine.session.run(machine.model.Smdl, machine.data.source)-1.

array([-0.065, -0.020, -0.032, ...,  0.010, -0.027,  0.047])

In [259]:
list(machine.model.J.eval(machine.session))

[0.20459527044081585,
 0.17447496695996442,
 0.33129932080427993,
 0.36228710028371003]

In [302]:
print(machine.session.run([machine.model.Fs,machine.model.F0],machine.data.source))

[array([-44.727, -44.724, -44.486, ..., -43.511, -43.956, -43.067]), -44.982257622410891]


In [303]:
len(machine.data.source[machine.model.Ssys])

1161

## Performance
### Implementing LogDet
Define logdet and its gradient

In [1]:
import numpy as np
import tensorflow as tf
# from https://gist.github.com/harpone/3453185b41d8d985356cbe5e57d67342
# Define custom py_func which takes also a grad op as argument:
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    tf.RegisterGradient(rnd_name)(grad)  # see _MySquareGrad for grad example
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
# from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/linalg_grad.py
# Gradient for logdet
def logdet_grad(op, grad):
    a = op.inputs[0]
    a_adj_inv = tf.matrix_inverse(a, adjoint=True)
    out_shape = tf.concat([tf.shape(a)[:-2], [1, 1]], axis=0)
    return tf.reshape(grad, out_shape) * a_adj_inv
# define logdet by calling numpy.linalg.slogdet
def logdet(a, name = None):
    with tf.name_scope(name, 'LogDet', [a]) as name:
        res = py_func(lambda a: np.linalg.slogdet(a)[1], 
                      [a], 
                      tf.float64, 
                      name=name, 
                      grad=logdet_grad) # set the gradient
        return res

test

In [2]:
sess = tf.Session()
v = tf.Variable(np.array([[4.,0.,0.,2.],[2.,1.,3.,4.]]))
T = tf.constant(np.array([[[1.,0.],[0.,0.]],[[0.,1.],[0.,0.]],[[0.,0.],[1.,0.]],[[0.,0.],[0.,1.]]]))
x = tf.tensordot(v, T, axes = 1)
y = logdet(x)
g = tf.gradients(y, [v])
sess.run(tf.global_variables_initializer())
sess.run([x, y, g])

[array([[[ 4.,  0.],
         [ 0.,  2.]],
 
        [[ 2.,  1.],
         [ 3.,  4.]]]),
 array([ 2.07944154,  1.60943791]),
 [array([[ 0.25,  0.  ,  0.  ,  0.5 ],
         [ 0.8 , -0.6 , -0.2 ,  0.4 ]])]]

In [4]:
i = tf.constant(np.eye(500))
x = tf.Variable(np.array([10.]))
y = logdet(x*i)
dy = tf.gradients(y, [x])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([y, dy]))

[1151.2925464970251, [array([ 50.])]]


### Benchmark

In [6]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=128), IsingModel(SSLattice(width = 16, depth = 5, pattern = [-1,-1,1])))
machine.initialize()

In [43]:
for i in range(2):
    print(machine.train(1, check = 1))

(0.0016886740341688002, array([ 0.275,  0.026,  0.012,  0.073]))
(0.001681922492786037, array([ 0.275,  0.024,  0.012,  0.073]))


In [29]:
list(machine.model.J.eval(machine.session))

[0.25357365033744755,
 0.13065293065882538,
 0.093628131393413694,
 0.055901301696411297]

In [45]:
machine.session.run(machine.model.J.assign(
        [0.25357365033744755,
         0.13065293065882538,
         0.093628131393413694,
         0.055901301696411297]))

array([ 0.254,  0.131,  0.094,  0.056])

In [46]:
machine.session.run(machine.model.MSE, machine.data.source)

0.0024871075361979638

In [7]:
machine.writer.add_graph(machine.graph) 